[View in Colaboratory](https://colab.research.google.com/github/xidulu/Naive_CNN/blob/master/main.ipynb)

In [0]:
import keras
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import np_utils

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

观察到原始数据维度：28x28

In [98]:
print("size:",len(x_train[1]),len(x_train[1][0]))

size: 28 28


In [58]:
print(x_train[0])
print(y_train)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 18

对数据集进行观察后，发现图像是由灰度数组表示的，范围是$[0,255]$。对X采取归一化，归至为$[0,1]$。对于Y，原数据直接用数字进行标注，不便于进行学习，因此转换为one-hot的形式，使用np_utils.to_categorical()函数。

In [0]:
row, col = 28, 28

In [0]:
#@title 归一化处理
X_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
X_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')
X_train = X_train / 255
X_test = X_test / 255

In [0]:
#@title 转换为one-hot
Y_train = np_utils.to_categorical(y_train)
Y_test = np_utils.to_categorical(y_test)

In [0]:
num_classes = Y_train.shape[1]

In [78]:
X_train.shape

(60000, 28, 28, 1)

这里我使用了卷积神经网络，共5层

1.   输入层：28x28x1
2.   卷积层：filter size 32，kernel size 5x5, 激活函数为ReLU
3.   池化层(2x2)
4.   dropout layer减少过拟合
5.   全连接层，使用ReLU，128个神经元，用于计算卷积层生成的特征对应的权重
6.   用softmax多重分类器，生成输出层！

metrics使用准确率，loss function使用交叉熵（因为是多重分类问题），参数初始方法采用keras默认的Xavier Initialization.


In [0]:
def cnn_model():
  model = Sequential()
  model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))
  #Fully connected layer
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))
	# Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
model = cnn_model()

In [101]:
model.fit(X_train, Y_train, validation_data = (X_test, Y_test), epochs=20, batch_size=256, verbose = 1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s 60us/step - loss: 0.0020 - acc: 0.9994 - val_loss: 0.0350 - val_acc: 0.9919
Epoch 2/20
60000/60000 [==============================] - 4s 61us/step - loss: 0.0041 - acc: 0.9987 - val_loss: 0.0416 - val_acc: 0.9901
Epoch 3/20
60000/60000 [==============================] - 4s 62us/step - loss: 0.0042 - acc: 0.9983 - val_loss: 0.0442 - val_acc: 0.9911
Epoch 4/20
60000/60000 [==============================] - 4s 62us/step - loss: 0.0039 - acc: 0.9989 - val_loss: 0.0375 - val_acc: 0.9908
Epoch 5/20
18944/60000 [========>.....................] - ETA: 2s - loss: 0.0029 - acc: 0.9988

60000/60000 [==============================] - 4s 62us/step - loss: 0.0032 - acc: 0.9989 - val_loss: 0.0375 - val_acc: 0.9915
Epoch 6/20
60000/60000 [==============================] - 4s 62us/step - loss: 0.0022 - acc: 0.9993 - val_loss: 0.0387 - val_acc: 0.9911
Epoch 7/20
60000/60000 [==============================] - 4s 61us/step - loss: 0.0025 - acc: 0.9993 - val_loss: 0.0398 - val_acc: 0.9914
Epoch 8/20
60000/60000 [==============================] - 4s 62us/step - loss: 0.0019 - acc: 0.9995 - val_loss: 0.0415 - val_acc: 0.9913
Epoch 9/20
47104/60000 [======================>.......] - ETA: 0s - loss: 0.0048 - acc: 0.9984

60000/60000 [==============================] - 4s 62us/step - loss: 0.0041 - acc: 0.9987 - val_loss: 0.0384 - val_acc: 0.9913
Epoch 10/20
60000/60000 [==============================] - 4s 62us/step - loss: 0.0033 - acc: 0.9989 - val_loss: 0.0420 - val_acc: 0.9912
Epoch 11/20
60000/60000 [==============================] - 4s 61us/step - loss: 0.0030 - acc: 0.9991 - val_loss: 0.0422 - val_acc: 0.9911
Epoch 12/20
60000/60000 [==============================] - 4s 62us/step - loss: 0.0020 - acc: 0.9994 - val_loss: 0.0341 - val_acc: 0.9924
Epoch 13/20
49664/60000 [=======================>......] - ETA: 0s - loss: 0.0015 - acc: 0.9996

60000/60000 [==============================] - 4s 61us/step - loss: 0.0020 - acc: 0.9994 - val_loss: 0.0406 - val_acc: 0.9911
Epoch 14/20
60000/60000 [==============================] - 4s 62us/step - loss: 0.0021 - acc: 0.9993 - val_loss: 0.0397 - val_acc: 0.9894
Epoch 15/20
60000/60000 [==============================] - 4s 62us/step - loss: 0.0013 - acc: 0.9997 - val_loss: 0.0395 - val_acc: 0.9912
Epoch 16/20
60000/60000 [==============================] - 4s 62us/step - loss: 0.0018 - acc: 0.9995 - val_loss: 0.0413 - val_acc: 0.9913
Epoch 17/20
50688/60000 [========================>.....] - ETA: 0s - loss: 0.0029 - acc: 0.9991

60000/60000 [==============================] - 4s 62us/step - loss: 0.0030 - acc: 0.9990 - val_loss: 0.0420 - val_acc: 0.9904
Epoch 18/20
60000/60000 [==============================] - 4s 61us/step - loss: 0.0026 - acc: 0.9993 - val_loss: 0.0395 - val_acc: 0.9923
Epoch 19/20
60000/60000 [==============================] - 4s 62us/step - loss: 0.0021 - acc: 0.9994 - val_loss: 0.0450 - val_acc: 0.9911
Epoch 20/20
60000/60000 [==============================] - 4s 63us/step - loss: 0.0020 - acc: 0.9994 - val_loss: 0.0423 - val_acc: 0.9910


In [102]:
print(model.evaluate(X_test, Y_test, verbose=0))

[0.04232813151813334, 0.991]


在测试集上，运行结果为：准确率$0.991$